In [9]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy.stats as st
from IPython.display import display, Markdown
from scipy.optimize import fsolve
from sklearn.linear_model import LinearRegression

from common_plotting import read_data, plot, years, months, yearsFmt, add_thp
from common_analysis import analyse_grouped_ohms, PT385_eq, Trh, Rt, Rs23, add_dut_and_setting_group, interpolate_temp

plt.rcParams["figure.figsize"] = (20,10)
orig_max_rows = pd.options.display.max_rows
pd.options.display.max_rows = 100

In [24]:
thp_data, data = read_data(['ks3458a-f5450a-sweep.csv', 'ks3458a-f5450a-sweep-with-1.9.csv', 'ks3458a-d4700-resistance-sweep.csv', 'ks3458a1-f5450a-sweep-with-1.9.csv', 'ks3458a1-d4700-resistance-sweep.csv', 'ks3458a-f5450a-best-resistors-comparison.csv'])
ks3458a_f5450a_sweep = data['ks3458a-f5450a-sweep.csv']
ks3458a_f5450a_sweep_with_19 = data['ks3458a-f5450a-sweep-with-1.9.csv']
ks3458a_d4700_resistance_sweep = data['ks3458a-d4700-resistance-sweep.csv']
ks3458a1_f5450a_sweep_with_19 = data['ks3458a1-f5450a-sweep-with-1.9.csv']
ks3458a1_d4700_resistance_sweep = data['ks3458a1-d4700-resistance-sweep.csv']
ks3458a_f5450a_best_resistors_comparison = data['ks3458a-f5450a-best-resistors-comparison.csv']

In [25]:
ks3458a_f5450a_sweep_1 = ks3458a_f5450a_sweep[
    (ks3458a_f5450a_sweep.index > pd.Timestamp(2023, 1, 25, 14, 55))
    & (ks3458a_f5450a_sweep.index < pd.Timestamp(2023, 1, 27, 0))
].copy()
ks3458a_f5450a_sweep_2 = ks3458a_f5450a_sweep[
    (ks3458a_f5450a_sweep.index > pd.Timestamp(2023, 1, 27, 0))
    & (ks3458a_f5450a_sweep.index < pd.Timestamp(2023, 2, 16, 0))
].copy()
ks3458a_f5450a_sweep_with_19_1 = ks3458a_f5450a_sweep_with_19[
    (ks3458a_f5450a_sweep_with_19.index > pd.Timestamp(2023, 1, 28, 0))
    & (ks3458a_f5450a_sweep_with_19.index < pd.Timestamp(2023, 1, 30, 0))
].copy()
ks3458a_f5450a_sweep_with_19_2 = ks3458a_f5450a_sweep_with_19[
    (ks3458a_f5450a_sweep_with_19.index > pd.Timestamp(2023, 1, 30, 0))
    & (ks3458a_f5450a_sweep_with_19.index < pd.Timestamp(2023, 2, 3, 0))
].copy()
ks3458a_f5450a_sweep_with_19_3 = ks3458a_f5450a_sweep_with_19[
    (ks3458a_f5450a_sweep_with_19.index > pd.Timestamp(2023, 2, 21, 0))
    & (ks3458a_f5450a_sweep_with_19.index < pd.Timestamp(2023, 2, 23, 0))
].copy()
ks3458a_d4700_resistance_sweep_1 = ks3458a_d4700_resistance_sweep[
    (ks3458a_d4700_resistance_sweep.index > pd.Timestamp(2023, 1, 26, 0))
    & (ks3458a_d4700_resistance_sweep.index < pd.Timestamp(2023, 1, 28, 0))
].copy()
ks3458a_d4700_resistance_sweep_2 = ks3458a_d4700_resistance_sweep[
    (ks3458a_d4700_resistance_sweep.index > pd.Timestamp(2023, 2, 2, 0))
    & (ks3458a_d4700_resistance_sweep.index < pd.Timestamp(2023, 2, 4, 0))
].copy()
ks3458a_d4700_resistance_sweep_3 = ks3458a_d4700_resistance_sweep[
    (ks3458a_d4700_resistance_sweep.index > pd.Timestamp(2023, 2, 21, 0))
    & (ks3458a_d4700_resistance_sweep.index < pd.Timestamp(2023, 2, 22, 0))
].copy()
ks3458a1_f5450a_sweep_with_19_1 = ks3458a1_f5450a_sweep_with_19[
    (ks3458a1_f5450a_sweep_with_19.index > pd.Timestamp(2023, 2, 2, 0))
    & (ks3458a1_f5450a_sweep_with_19.index < pd.Timestamp(2023, 2, 4, 0))
].copy()
ks3458a1_f5450a_sweep_with_19_2 = ks3458a1_f5450a_sweep_with_19[
    (ks3458a1_f5450a_sweep_with_19.index > pd.Timestamp(2023, 2, 21, 0))
    & (ks3458a1_f5450a_sweep_with_19.index < pd.Timestamp(2023, 2, 22, 0))
].copy()
ks3458a1_d4700_resistance_sweep_1 = ks3458a1_d4700_resistance_sweep[
    (ks3458a1_d4700_resistance_sweep.index > pd.Timestamp(2023, 2, 1, 0))
    & (ks3458a1_d4700_resistance_sweep.index < pd.Timestamp(2023, 2, 3, 0))
].copy()
ks3458a1_d4700_resistance_sweep_2 = ks3458a1_d4700_resistance_sweep[
    (ks3458a1_d4700_resistance_sweep.index > pd.Timestamp(2023, 2, 21, 0))
    & (ks3458a1_d4700_resistance_sweep.index < pd.Timestamp(2023, 2, 23, 0))
].copy()


def analyse_ohms_sweep(data, meter):
    meter_number = meter.split("_")[1]
    interpolate_temp(data, f"temp_{meter_number}")
    interpolated_data_without_temp_rows = data[data[f"temp_{meter_number}"].isna()]
    group_columns = ["dut", "dut_setting", f"{meter}_range"]
    groups = (
        (
            interpolated_data_without_temp_rows[group_columns].apply(tuple, axis=1)
            != interpolated_data_without_temp_rows[group_columns]
            .shift()
            .apply(tuple, axis=1)
        )
        .cumsum()
        .rename("group")
    )
    grouped_data = interpolated_data_without_temp_rows.join(groups)
    return analyse_grouped_ohms(
        grouped_data,
        meter=meter,
        temperature_columns=[f"temp_{meter_number}_interpolated", "temperature"],
    )


ks3458a_f5450a_sweep_1_summary = analyse_ohms_sweep(ks3458a_f5450a_sweep_1, "ag3458a_2")
ks3458a_f5450a_sweep_2_summary = analyse_ohms_sweep(ks3458a_f5450a_sweep_2, "ag3458a_2")
ks3458a_f5450a_sweep_with_19_summary_1 = analyse_ohms_sweep(ks3458a_f5450a_sweep_with_19_1, "ag3458a_2")
ks3458a_f5450a_sweep_with_19_summary_2 = analyse_ohms_sweep(ks3458a_f5450a_sweep_with_19_2, "ag3458a_2")
ks3458a_f5450a_sweep_with_19_summary_3 = analyse_ohms_sweep(ks3458a_f5450a_sweep_with_19_3, "ag3458a_2")
ks3458a_d4700_resistance_sweep_summary_1 = analyse_ohms_sweep(ks3458a_d4700_resistance_sweep_1, "ag3458a_2")
ks3458a_d4700_resistance_sweep_summary_2 = analyse_ohms_sweep(ks3458a_d4700_resistance_sweep_2, "ag3458a_2")
ks3458a_d4700_resistance_sweep_summary_3 = analyse_ohms_sweep(ks3458a_d4700_resistance_sweep_3, "ag3458a_2")
ks3458a1_f5450a_sweep_with_19_summary_1 = analyse_ohms_sweep(ks3458a1_f5450a_sweep_with_19_1, "ag3458a_1")
ks3458a1_f5450a_sweep_with_19_summary_2 = analyse_ohms_sweep(ks3458a1_f5450a_sweep_with_19_2, "ag3458a_1")
ks3458a1_d4700_resistance_sweep_summary_1 = analyse_ohms_sweep(ks3458a1_d4700_resistance_sweep_1, "ag3458a_1")
ks3458a1_d4700_resistance_sweep_summary_2 = analyse_ohms_sweep(ks3458a1_d4700_resistance_sweep_2, "ag3458a_1")
ks3458a_f5450a_best_resistors_comparison_summary = analyse_ohms_sweep(ks3458a_f5450a_best_resistors_comparison, "ag3458a_2")

In [32]:
ks3458a_f5450a_best_resistors_comparison_summary

,dut,dut_setting,count,ohm_mean,ohm_sem,ohm_std,range,temp_2_interpolated_mean,temperature_mean
group,,,,,,,,,
1,Fluke 5450A,10 kOhm,16,10000.368786,0.000404,0.001616,10000.0,NaN,33.191711
2,Fluke 5450A,short,16,0.000084,0.000122,0.000489,10000.0,51.137037,33.077930
3,SR104,10 kOhm,16,9999.991219,0.000337,0.001349,10000.0,51.229630,32.951489
4,Measurements International 9331,10 kOhm,16,9999.815113,0.000272,0.001087,10000.0,51.322975,32.879980
5,Guildline 9330,10 kOhm,16,10000.437514,0.000414,0.001655,10000.0,51.373438,32.828198
6,SR104,10 kOhm,16,9999.992688,0.000356,0.001423,10000.0,51.354054,32.966919
7,Fluke 5450A,10 kOhm,16,10000.375306,0.000498,0.001992,10000.0,51.267568,33.206970
8,Fluke 5450A,1 kOhm,16,1000.066878,0.000193,0.000772,10000.0,51.203378,33.247986
9,Fluke 5450A,1 kOhm,16,1000.066323,0.000030,0.000121,1000.0,51.200000,33.047412
